In [1]:
# import importlib
# importlib.reload(cluster_util)
# import cluster_util

# Notebook error analysis 

## Refining error types - Evaluate cluster results

We implemented and tried alternative embeddings and clustering algorithms. We focus on:

    embeddings:
        document level embedding -> sentence transformer
        word level embedding -> finetuned word embedding with fasttext (support subword)
    clustering:
        dbscan
        optics
Evaluate clusters:

    take a random subset (say 20 errors), take each pair, manually label -> if they should be in the same cluster or not
    check clustering results based on the labeling
    then choose a reasonable embedding+clustering technique
(skip)ML bug categorization:

    with the chosen embedding+clustering results, summarize each clusters and assign them to existing or new categories

In [1]:
# clustered results

import pandas as pd 
import cluster_util, config

df_mlerr_clustered = pd.read_excel(config.path_default.joinpath("nlpproject/df_err_processed_pregroup_dedup_clustered.xlsx"))

C:\Users\yirwa29\AppData\Local\anaconda3\envs\chatgpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


path_default=WindowsPath('C:/Users/yirwa29/Downloads/Dataset-Nb')


In [85]:
import util
# util.print_traceback(df_mlerr_clustered.iloc[51].traceback)
df_mlerr_clustered.iloc[51]

fname                                              00003-0-ann-experiment1.ipynb
eid                                         45ca9130-c1f3-3a2e-9b2c-b745bb480d48
ename                                                                  typeerror
evalue                                           Perceptron() takes no arguments
traceback                      ['--------------------------------------------...
ename_mapped                                                           typeerror
imports                        {('', 'numpy', 'np'), ('', 'matplotlib.pyplot'...
lib_alias                      [['numpy', 'np'], ['matplotlib', 'plt'], ['skl...
is_MLnb                                                                     True
is_relevant                                                                 True
if_ast_py23                                                                  1.0
python_version                                                                 3
if_ast                      

In [2]:
df_mlerr_clustered.head()

,fname,eid,ename,evalue,traceback,ename_mapped,imports,lib_alias,is_MLnb,is_relevant,...,if_ast,nb_source,evalue_processed,evalue_tokenized,hash_evalue_tokenized,pregroup_cluster,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb
0,00000-1171-strax-demo.ipynb,b63153b4-9eee-385e-b788-c093678c5068,datanotavailable,Time range selection assumes data is already a...,['--------------------------------------------...,libspecificerrors,"{('', 'matplotlib.pyplot', 'plt'), ('', 'strax...","[['matplotlib', 'plt'], ['strax', 'strax'], ['...",True,True,...,1,2,time range selection assumes data is already a...,time range selection assumes data already avai...,1ef06b829d3175d7c1fac1a92cd4a1df,1,0,0,0,-1
1,00000-2217-visual-turing-test.ipynb,8344b7ca-3454-3a42-8308-5b7c8da1836f,runtimeerror,Cuda error: CudaNdarray_TakeFrom: invalid devi...,['--------------------------------------------...,runtimeerror,"{('kraino.core.model_zoo', 'Config', ''), ('kr...","[['kraino', 'Config'], ['kraino', 'DropMask'],...",True,True,...,1,2,cuda error invalid device function,cuda error cudandarraytakefrom invalid device ...,b3ef747f4f8f1a4ecbf75efa01402e94,11,0,369,0,568
2,00000-2353-student-project.ipynb,ff8fcb95-97a2-3df1-8649-cd9a7da6c866,attributeerror,module 'tfx_bsl.arrow.array_util' has no attri...,['--------------------------------------------...,attributeerror,"{('aequitas.plotting', 'Plot', ''), ('functool...","[['aequitas', 'Plot'], ['functools', 'partial'...",True,True,...,1,2,module has no attribute while running,module attribute valuecounts running generates...,c640359d9be8c54aec809c8d5c595043,13,0,318,0,-1
3,00000-246-fasttext-6th-step.ipynb,d1c6f1c5-4839-3233-b523-1fb7153aade1,attributeerror,probability estimates are not available for lo...,['--------------------------------------------...,attributeerror,"{('sklearn.metrics', 'plot_confusion_matrix', ...","[['sklearn', 'plot_confusion_matrix'], ['scipy...",True,True,...,1,2,probability estimates are not available for loss,probability estimates available loss hinge,2b8682f69ca05d2e666304e872098bd2,15,0,-1,0,-1
4,00000-2486-encode-decode.ipynb,c4e051d2-3f40-3b3d-84e1-feab49fae21f,valueerror,read of closed file,['--------------------------------------------...,valueerror,"{('', 'cv2', ''), ('', 'matplotlib.pyplot', 'p...","[['cv2', 'cv2'], ['matplotlib', 'plt'], ['PIL'...",True,True,...,1,2,read of closed file,read closed file,04823468ba0a924ff70eb8c7382ddfc6,16,0,515,1,698


In [17]:
# random sample pairs
# sample 0: random
# sample 1: exclude those are labeled as noises by any of the clustering method
# sample 2: sample 1 constraint + valueerror

import random

check_columns = ['cluster_dbscan_transformers', 'cluster_optics_transformers',
                 'cluster_dbscan_wordemb', 'cluster_optics_wordemb']

def noise_checker(row):
    for cn in check_columns:
        if row[cn] == -1:
            return True
    return False

def exp_checker(row, exp_name="valueerror"):
    return row["ename"] == exp_name

# all pairs
# exclude ones that are noises from any of the clustering results
id_pairs = []
for i in df_mlerr_clustered.index:
    if noise_checker(df_mlerr_clustered.loc[i])==False and exp_checker(df_mlerr_clustered.loc[i]):
        for j in range(i+1, len(df_mlerr_clustered.index)):
            if noise_checker(df_mlerr_clustered.loc[j])==False and exp_checker(df_mlerr_clustered.loc[j]):
                id_pairs.append((i, j))
print(len(id_pairs))

# random sample
n = 20
sampled_pair_ids = random.sample(id_pairs, n)

# get dataframes
sampled_pair_id_left = [p[0] for p in sampled_pair_ids]
sampled_pair_id_right = [p[1] for p in sampled_pair_ids]

df_mlerr_sample_left = df_mlerr_clustered.loc[sampled_pair_id_left,:]
df_mlerr_sample_right = df_mlerr_clustered.loc[sampled_pair_id_right,:]

# save the sampled dataframes
df_mlerr_sample_left.to_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_left.xlsx"),
                              index=False, engine='xlsxwriter')
df_mlerr_sample_right.to_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_right.xlsx"),
                               index=False, engine='xlsxwriter')

41328


In [18]:
df_mlerr_sample_left

,fname,eid,ename,evalue,traceback,ename_mapped,imports,lib_alias,is_MLnb,is_relevant,...,if_ast,nb_source,evalue_processed,evalue_tokenized,hash_evalue_tokenized,pregroup_cluster,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb
3179,00187-3127-3-model-design.ipynb,878012bc-ad48-3f10-b2a3-8d5b02fd55fa,valueerror,No gradients provided for any variable: (['den...,['--------------------------------------------...,valueerror,"{('sklearn.metrics', 'recall_score', ''), ('te...","[['sklearn', 'recall_score'], ['tensorflow', '...",True,True,...,1,2,no gradients provided for any variable provide...,no gradients provided variable provided gradsa...,368e2e206bcdeab36d8c8b235e088552,3877,0,290,0,155
298,00018-774-integrating-specutils.ipynb,ef87c0c5-65f7-3e76-a3d8-7a287bcafa31,valueerror,'1E-17 erg/cm^2/s/Ang' did not parse as unit: ...,['--------------------------------------------...,valueerror,"{('astropy', 'units', ''), ('pkg_resources', '...","[['astropy', 'units'], ['pkg_resources', 'reso...",True,True,...,1,2,did not parse as unit at col ang is not a vali...,erg cm ang parse unit at col ang valid unit di...,174501560816b08b9785ba2ea260665f,483,0,775,0,303
5516,00283-2742-regression.ipynb,cff407d8-9770-32c8-b74d-b4d7aae14079,valueerror,cannot call `vectorize` with a signature inclu...,['--------------------------------------------...,valueerror,"{('xarrayutils.utils', '_linregress_ufunc', ''...","[['xarrayutils', '_linregress_ufunc'], ['scipy...",True,True,...,1,2,cannot call vectorize with a signature includi...,call vectorize signature including new output ...,57dabe7569ceebb3855de5dca43ebad8,6024,0,516,0,240
3157,00186-3109-mnist-gan-exercise.ipynb,04061f5d-0953-37b8-b398-7559d5f09a87,valueerror,slice step cannot be zero,['--------------------------------------------...,valueerror,"{('', 'matplotlib.pyplot', 'plt'), ('', 'torch...","[['matplotlib', 'plt'], ['torch', 'optim'], ['...",True,True,...,1,2,slice step cannot be zero,slice step zero,223649d8c722330f2a83087daf0429e3,3853,0,560,0,617
3546,00205-1214-case2-biodiversity-processing.ipynb,83b17231-8b96-3201-b285-4ba389d64a48,valueerror,cannot assemble the datetimes: day is out of r...,['--------------------------------------------...,valueerror,"{('', 'mplleaflet', ''), ('', 'matplotlib.pypl...","[['mplleaflet', 'mplleaflet'], ['matplotlib', ...",True,True,...,1,2,cannot assemble the datetimes day is out of ra...,assemble datetimes day range month,01c02d0605be009539ddd4bdaa982d84,4237,0,2,0,249
1293,00081-599-kepler-eb.ipynb,19025890-03d8-3b04-9d5c-a26401167dc5,valueerror,No table found,['--------------------------------------------...,valueerror,"{('', 'matplotlib', 'mpl'), ('', 'matplotlib.c...","[['matplotlib', 'mpl'], ['matplotlib', 'colors...",True,True,...,1,2,no table found,no table found,c4bf1bc4282fcaf69ddf3dc1983bb402,1803,0,186,0,657
3546,00205-1214-case2-biodiversity-processing.ipynb,83b17231-8b96-3201-b285-4ba389d64a48,valueerror,cannot assemble the datetimes: day is out of r...,['--------------------------------------------...,valueerror,"{('', 'mplleaflet', ''), ('', 'matplotlib.pypl...","[['mplleaflet', 'mplleaflet'], ['matplotlib', ...",True,True,...,1,2,cannot assemble the datetimes day is out of ra...,assemble datetimes day range month,01c02d0605be009539ddd4bdaa982d84,4237,0,2,0,249
1415,00089-35-iris-xgboost.ipynb,29547ffb-84af-3a88-ac61-a50e86d7abca,valueerror,The number of classes in labels is different f...,['--------------------------------------------...,valueerror,"{('', 'pandas', ''), ('sklearn.preprocessing',...","[['pandas', 'pandas'], ['sklearn', 'LabelEncod...",True,True,...,1,2,the number of classes in labels is different f...,the number classes labels different ypred clas...,8fbdba79c359c64bb8c99584fb62c084,1953,0,473,0,152
539,00033-479-cleaning-checkpoint.ipynb,bbe1ee21-60d4-3439-a4ae-552fd450118b,valueerror,NaTType does not support strftime,['--------------------------------------------...,

In [19]:
import numpy as np

# gather results from clustering algorithms
res = {} # same:1, not same:0
for cn in check_columns:
    res[cn] = np.full(n, -1)
    for i in range(n):
        if df_mlerr_sample_left.iloc[i][cn] != -1 or df_mlerr_sample_right.iloc[i][cn] != -1:
            res[cn][i] = df_mlerr_sample_left.iloc[i][cn]==df_mlerr_sample_right.iloc[i][cn]
        
reserve_columns = ['fname', 'eid', 'ename', 'evalue', 'traceback']

df_res_mlerr_sample = pd.DataFrame.from_dict(res)
for cn in reserve_columns:
    df_res_mlerr_sample.loc[:, cn+"_left"] = df_mlerr_sample_left[cn].reset_index(drop=True)
    df_res_mlerr_sample.loc[:, cn+"_right"] = df_mlerr_sample_right[cn].reset_index(drop=True)
df_res_mlerr_sample.to_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_res_algrithms.xlsx"),
                             index=False, engine='xlsxwriter')

In [20]:
df_res_mlerr_sample

,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb,fname_left,fname_right,eid_left,eid_right,ename_left,ename_right,evalue_left,evalue_right,traceback_left,traceback_right
0,1,0,1,0,00187-3127-3-model-design.ipynb,00311-41-animation-ahlive.ipynb,878012bc-ad48-3f10-b2a3-8d5b02fd55fa,3f69c8aa-ac26-37f8-acb6-a92d28404f28,valueerror,valueerror,No gradients provided for any variable: (['den...,conflicting sizes for dimension 'state': lengt...,['--------------------------------------------...,['--------------------------------------------...
1,1,0,1,0,00018-774-integrating-specutils.ipynb,00267-1814-word-analysis.ipynb,ef87c0c5-65f7-3e76-a3d8-7a287bcafa31,2b50ea49-8e57-39ae-817a-c2fd0b37779e,valueerror,valueerror,'1E-17 erg/cm^2/s/Ang' did not parse as unit: ...,a must be greater than 0,['--------------------------------------------...,['--------------------------------------------...
2,1,0,1,0,00283-2742-regression.ipynb,mohamedaziz15_regressor-cost.ipynb,cff407d8-9770-32c8-b74d-b4d7aae14079,fd45fe0c-128c-3aa7-bb81-e70b8b4f0164,valueerror,valueerror,cannot call `vectorize` with a signature inclu...,Input X contains NaN.,['--------------------------------------------...,['--------------------------------------------...
3,1,0,1,0,00186-3109-mnist-gan-exercise.ipynb,susuyue_notebookc18c69f749.ipynb,04061f5d-0953-37b8-b398-7559d5f09a87,1c2f2986-3d90-3614-b7d7-11695bc32eb9,valueerror,valueerror,slice step cannot be zero,"Number of rows must be a positive integer, not 0",['--------------------------------------------...,['--------------------------------------------...
4,1,0,1,0,00205-1214-case2-biodiversity-processing.ipynb,00279-624-highway-env-checkpoint.ipynb,83b17231-8b96-3201-b285-4ba389d64a48,985b91b8-2285-3fba-b0f3-609a8c886c55,valueerror,valueerror,cannot assemble the datetimes: day is out of r...,Error: the file her_sac_highway could not be f...,['--------------------------------------------...,['--------------------------------------------...
5,1,0,0,0,00081-599-kepler-eb.ipynb,00152-2862-quantile-plots.ipynb,19025890-03d8-3b04-9d5c-a26401167dc5,2eacaf5f-ca32-3367-8145-eefab9305bdc,valueerror,valueerror,No table found,"Illegal format string ""br-""; two color symbols",['--------------------------------------------...,['--------------------------------------------...
6,1,0,0,0,00205-1214-case2-biodiversity-processing.ipynb,00290-3083-4-0-port-toms-network-to-tf-keras.i...,83b17231-8b96-3201-b285-4ba389d64a48,8460064a-15da-35ac-b599-46d1940bcee2,valueerror,valueerror,cannot assemble the datetimes: day is out of r...,Unknown layer: SurRadLayer,['--------------------------------------------...,['--------------------------------------------...
7,1,0,1,0,00089-35-iris-xgboost.ipynb,rsd181234_food-classifier-mobilenetv2-resnet50...,29547ffb-84af-3a88-ac61-a50e86d7abca,7342a2b5-89b5-36b7-9ad7-48b476ad7203,valueerror,valueerror,The number of classes in labels is different f...,This model has not yet been built. Build the m...,['--------------------------------------------...,['--------------------------------------------...
8,1,0,1,0,00033-479-cleaning-checkpoint.ipynb,00271-1341-cholorpeth-checkpoint.ipynb,bbe1ee21-60d4-3439-a4ae-552fd450118b,709abe08-7cd2-333e-9bd0-65e60dcc68dc,valueerror,valueerror,NaTType does not support strftime,dless_start_perc encoding field is specified w...,['--------------------------------------------...,['--------------------------------------------...
9,1,0,1,0,00166-1960-0-ground-truth-processing.ipynb,00261-1208-1-0-sm-initial-eda.ipynb,9036533c-d6a1-35e8-be37-7ea462d76534,032893f0-a050-3694-bc38-5ab56a2efa97,valueerror,valueerror,array not found at path 'masked',"labels_true must be 1D: shape is (8580, 126355)",['--------------------------------------------...,['--------------------------------------------...


### Compare with manual results

In [21]:
# sampled clustered results

import pandas as pd 
import cluster_util, config

check_columns = ['cluster_dbscan_transformers', 'cluster_optics_transformers',
                 'cluster_dbscan_wordemb', 'cluster_optics_wordemb']

df_res_mlerr_sample0 = pd.read_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_res_algrithms0_random.xlsx"))
df_res_mlerr_sample1 = pd.read_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_res_algrithms1_nonoise.xlsx"))
df_res_mlerr_sample2 = pd.read_excel(config.path_default.joinpath("nlpproject/df_mlerr_clustered_sample_res_algrithms2_nonoise_valueerror.xlsx"))

In [36]:
# very imbalanced -> dedupped errors are not likely to represent similar things
df_res_mlerr_sample0.manual_label.value_counts()

manual_label
0    19
1     1
Name: count, dtype: int64

In [37]:
df_res_mlerr_sample1.manual_label.value_counts()

manual_label
0    20
Name: count, dtype: int64

In [38]:
df_res_mlerr_sample2.manual_label.value_counts()

manual_label
0    18
1     2
Name: count, dtype: int64

In [33]:
arr_eval = np.zeros((3,4))
name_cols = check_columns
name_rows = ['sample (random)','sample (without noise)','sample (without noise value errors)']

for i in range(len(check_columns)):
    cn = check_columns[i]
    # noise considered as separate clusters
    arr_eval[0, i] = sum((df_res_mlerr_sample0["manual_label"]==df_res_mlerr_sample0[cn]) | 
                         ((df_res_mlerr_sample0[cn]==-1) & (df_res_mlerr_sample0["manual_label"]==0)))/len(df_res_mlerr_sample0)
    arr_eval[1, i] = sum(df_res_mlerr_sample1["manual_label"]==df_res_mlerr_sample1[cn])/len(df_res_mlerr_sample1)
    arr_eval[2, i] = sum(df_res_mlerr_sample2["manual_label"]==df_res_mlerr_sample2[cn])/len(df_res_mlerr_sample2)

pd.DataFrame(arr_eval, index=name_rows, columns=name_cols)

,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb
sample (random),0.2,0.95,0.40,0.95
sample (without noise),0.2,1.00,0.20,1.00
sample (without noise value errors),0.2,0.90,0.35,0.90


In [43]:
from sklearn.metrics import f1_score

arr_eval_f1 = np.zeros((3,4))

for i in range(len(check_columns)):
    cn = check_columns[i]
    # noise considered as separate clusters
    df_res_mlerr_sample0.loc[df_res_mlerr_sample0[cn]==-1,cn] = 0
    arr_eval_f1[0, i] = f1_score(df_res_mlerr_sample0["manual_label"], df_res_mlerr_sample0[cn], zero_division=np.nan)
    arr_eval_f1[1, i] = f1_score(df_res_mlerr_sample1["manual_label"], df_res_mlerr_sample1[cn], zero_division=np.nan)
    arr_eval_f1[2, i] = f1_score(df_res_mlerr_sample2["manual_label"], df_res_mlerr_sample2[cn], zero_division=np.nan)

pd.DataFrame(arr_eval_f1, index=name_rows, columns=name_cols)

,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb
sample (random),0.111111,0.0,0.142857,0.0
sample (without noise),0.000000,NaN,0.000000,NaN
sample (without noise value errors),0.200000,0.0,0.235294,0.0


In [35]:
# how much they agree with each other based on random sample
arr_eval = np.zeros((4,4))

for i in range(len(check_columns)):
    for j in range(len(check_columns)):
        cni = check_columns[i]
        cnj = check_columns[j]
        # noise considered as separate clusters
        arr_eval[i, j] = sum((df_res_mlerr_sample0[cni]==df_res_mlerr_sample0[cnj]) | 
                             ((df_res_mlerr_sample0[cni]==-1)&(df_res_mlerr_sample0[cnj]==0)) |
                             ((df_res_mlerr_sample0[cni]==0)&(df_res_mlerr_sample0[cnj]==-1)))/len(df_res_mlerr_sample0)

pd.DataFrame(arr_eval, index=check_columns, columns=check_columns)

,cluster_dbscan_transformers,cluster_optics_transformers,cluster_dbscan_wordemb,cluster_optics_wordemb
cluster_dbscan_transformers,1.00,0.15,0.50,0.15
cluster_optics_transformers,0.15,1.00,0.35,1.00
cluster_dbscan_wordemb,0.50,0.35,1.00,0.35
cluster_optics_wordemb,0.15,1.00,0.35,1.00
